# Types of Sources

There are many types of sources you can input for openMC but the two primary ways that will be discussed in this tutorial are point sources and ring sources.

# Point sources

To perform a neutronics simulation a neutron source must also be defined.

This python notebook allows users to make a simple OpenMC point source and plot its energy, position and initial directions.

In [ ]:
import openmc
import openmc_source_plotter  # extends the openmc.IndependentSource class to include plotting functions

This first code block creates an isotropic point source with 14 MeV monoenergetic neutrons.

In [ ]:
# initialises a new source object
my_source = openmc.IndependentSource()

# sets the location of the source to x=0 y=0 z=0
my_source.space = openmc.stats.Point((0, 0, 0))

# sets the direction to isotropic
my_source.angle = openmc.stats.Isotropic()

# sets the energy distribution to 100% 14MeV neutrons
my_source.energy = openmc.stats.Discrete([14e6], [1])

this next code block runs openmc and creates a file containing information on the initial particles. The energy information is the extracted and plotted

In [ ]:
my_source.plot_source_energy()

The next code block creates an isotropic point source with a fission energy distribution.

In [ ]:
my_source_2 = openmc.IndependentSource()
my_source_2.space = openmc.stats.Point((0, 0, 0))
my_source_2.angle = openmc.stats.Isotropic()

# Documentation on the Watt distribution is here
# https://docs.openmc.org/en/stable/pythonapi/generated/openmc.data.WattEnergy.html
my_source_2.energy = openmc.stats.Watt(a=988000.0, b=2.249e-06)

my_source_2.plot_source_energy()

This code block creates an isotropic point source with a fusion energy distribution.

In [ ]:
my_source_3 = openmc.IndependentSource()
my_source_3.space = openmc.stats.Point((0, 0, 0))
my_source_3.angle = openmc.stats.Isotropic()

# Documentation on the Muir distribution is here
# https://docs.openmc.org/en/stable/pythonapi/generated/openmc.stats.muir.html
my_source_3.energy = openmc.stats.muir(e0=14080000.0, m_rat=5.0, kt=20000.0)

my_source_3.plot_source_energy()

The following code block plots the birth location of the neutrons from a 14 MeV monoenergetic point source.

In [ ]:
# Creates an isotropic point source with monoenergetic 14MeV neutrons
my_source_4 = openmc.IndependentSource()
my_source_4.space = openmc.stats.Point((0, 0, 0))
my_source_4.angle = openmc.stats.Isotropic()
my_source_4.energy = openmc.stats.Discrete([14e6], [1])

my_source_4.plot_source_position()

Finally, the following code block plots the birth direction of the neutrons from the same source.

In [ ]:
# Plots the direction of the neutrons
my_source_4.plot_source_direction()

# Ring sources

We will make use of the CylindricalIndependent source as described in the docs
https://docs.openmc.org/en/stable/pythonapi/generated/openmc.stats.CylindricalIndependent.html

The CylindricalIndependent source can make makes a ring shaped source with the corect inputs.
- The r value (distribution of radius values)
- The phi value (distribution of azimuthal angle values)
- The z value (distribution of z values on the z axis)

Each of these inputs must be a univariate probability disitribution.

OpenMC has several built in distributions for popular distributions such as Normal, Maxwell and others.

See the full list here https://docs.openmc.org/en/stable/pythonapi/stats.html#univariate-probability-distributions

In this example we are going to make use of the Uniform disitrbution by feel free to tinker with the differnt distributions an make your own source.

In [ ]:
import openmc
import openmc_source_plotter  # extends the openmc.IndependentSource class to include plotting functions

# initialises a new source object
my_source = openmc.IndependentSource()

# the distribution of radius is just a single value
radius = openmc.stats.Discrete([10], [1])

# the distribution of source z values is just a single value
z_values = openmc.stats.Discrete([0], [1])

# the distribution of source azimuthal angles values is a uniform distribution between 0 and 2 Pi
angle = openmc.stats.Uniform(a=0., b=2* 3.14159265359)

# this makes the ring source using the three distributions and a radius
my_source.space = openmc.stats.CylindricalIndependent(r=radius, phi=angle, z=z_values, origin=(0.0, 0.0, 0.0))

# sets the direction to isotropic
my_source.angle = openmc.stats.Isotropic()

# sets the energy distribution to a Muir distribution neutrons
my_source.energy = openmc.stats.muir(e0=14080000.0, m_rat=5.0, kt=20000.0)

this next code block runs openmc and creates a file containing information on the initial particles. The position information is the extracted and plotted

In [ ]:
my_source.plot_source_position()

This is closer to looking like a plasma compared to a point source but the distribution of spatial distribution can be improved in the next task

In [ ]:
# these plotting methods can also be run if you are interested in energy or direction
# my_source.plot_source_energy()
# my_source.plot_source_direction()

# Plasma source plotting

OpenMC can be used to create point sources with different energy distributions. However, there are other ways to create neutron sources for use in neutronics simulations.

This python notebook allows users to plot the energy, position and initial directions of a parametric plasma source.

In [ ]:
import openmc
import openmc_source_plotter  # extends the openmc.IndependentSource class to include plotting functions

This first code block creates a neutron source using the PlasmaSource class from the parametric_plasma_source package. The properties of the source are controlled by the input parameters.

In [ ]:
from openmc_plasma_source import TokamakSource

my_sources = TokamakSource(
    elongation=1.557,
    ion_density_centre=1.09e20,
    ion_density_peaking_factor=1,
    ion_density_pedestal=1.09e20,
    ion_density_separatrix=3e19,
    ion_temperature_centre=45.9,
    ion_temperature_peaking_factor=8.06,
    ion_temperature_pedestal=6.09,
    ion_temperature_separatrix=0.1,
    major_radius=9.06,
    minor_radius=2.92258,
    pedestal_radius=0.8 * 2.92258,
    mode="H",
    shafranov_factor=0.44789,
    triangularity=0.270,
    ion_temperature_beta=6,
    sample_size=50,  # the number of individual sources to use to make a combined source
    angles=(0, 2 * 3.14)  # angle in radians
).make_openmc_sources()  # returns a list of openmc sources

To plot the parametric plasma source we store the x y z birth locations, energies and directions of neutrons in the source in separate lists.

In [ ]:
# plots multiple sources on one plot by iterating through
# the list of sources and building up the plot
plot = None
for source in my_sources:
    plot = source.plot_source_energy(figure=plot)
plot.show()

This code block then plots the birth location of each neutron, coloured by neutron birth energy.

In [ ]:
# plots multiple sources on one plot by iterating through
# the list of sources and building up the plot
plot = None
for source in my_sources:
    plot = source.plot_source_position(figure=plot)
plot.show()

We can also plot the birth direction of each neutron.

In [ ]:
plot = None
for source in my_sources:
    plot = source.plot_source_direction(figure=plot)
plot.show()

**Learning Outcomes for Part 2:**

- Plasma sources can be defined using the parametric_plasma_source package.